In [2]:
from detectron2.modeling import build_model

/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dataclasses import dataclass
from detectron2.config import get_cfg
from config import get_settings
from ai.ditod import add_vit_config
@dataclass
class Arguments:
    dataset = "doclaynet"
    config_file_path = "ai/Configs/doclaynet_VGT_cascade_PTM.yaml"
    model_weights_path = "./models/model_final.pth"


args = Arguments()
cfg = get_cfg()
add_vit_config(cfg)
cfg.merge_from_file(args.config_file_path)


In [4]:
model = build_model(cfg)

use_pretrain_weight: load model from: ./models/layoutlm/


In [5]:
from detectron2.structures import ImageList, Instances
import torch
def preprocess_image(images):
    """
    Normalize, pad and batch the input images.
    """
    pixel_mean, pixel_std = torch.tensor([103.530, 116.280, 123.675]), torch.tensor([57.375, 57.120, 58.395])
    # images = [x["image"].to('cpu') for x in batched_inputs]
    images = [(x.permute(2,1,0) - pixel_mean) / pixel_std for x in images]
    images = [x.permute(2,1,0) for x in images]
    images = ImageList.from_tensors(
        images,
        0,
        padding_constraints={},
    )
    print(images.tensor.shape)
    return images

In [6]:
import torch 
import numpy as np
input_tensor = torch.rand(3,1000,1000)
input_dict = [{
    'image': input_tensor,
    'input_ids':torch.tensor([2,444,566]),
    'bbox': torch.tensor([[112,334,444,666], [23,35,666,435], [22,66,343,432]]),
    'height': torch.tensor(1000),
    'width': torch.tensor(1000)
}]

In [9]:
model.eval().to('cpu')

VGT(
  (backbone): GridFPN(
    (fpn_lateral2): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): PTM_VIT_Backbone(
      (backbone): BEiT(
        (patch_embed): PatchEmbed(
          (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        )
        (grid_patch_embed): PatchEmbed(
          (proj): Conv2d(64, 768, kernel_size=(16, 16), stride=(16, 16))
        )
 

In [10]:
torch_script = torch.jit.trace(model.forward, (input_dict, ))

/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/detectron2/structures/image_list.py:85: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert t.shape[:-2] == tensors[0].shape[:-2], t.shape
/home/saeed/Walnut/Quantization/ai/ditod/Wordnn_embedding.py:81: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  short_length_w = min(len(per_input_ids), len(per_input_bbox))
/home/saeed/Walnut/Quantization/ai/ditod/Wordnn_embedding.py:92: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be tre

In [ ]:
import torch
quant =torch.ao.quantization.QuantStub()
dequant = torch.ao.quantization.DeQuantStub()
x = torch.rand(2,2)
print(x)
quant(x)

tensor([[0.3068, 0.9300],
        [0.0045, 0.7737]])


tensor([[0.3068, 0.9300],
        [0.0045, 0.7737]])

In [7]:
from torch.onnx import OperatorExportTypes
import onnx
import io
def export_onnx_model(model, inputs):
    """
    Trace and export a model to onnx format.

    Args:
        model (nn.Module):
        inputs (tuple[args]): the model will be called by `model(*inputs)`

    Returns:
        an onnx model
    """
    assert isinstance(model, torch.nn.Module)

    # make sure all modules are in eval mode, onnx may change the training state
    # of the module if the states are not consistent
    def _check_eval(module):
        assert not module.training

    model.apply(_check_eval)

    # Export the model to ONNX
    with torch.no_grad():
        with io.BytesIO() as f:
            torch.onnx.export(
                model,
                inputs,
                f,
                operator_export_type=OperatorExportTypes.ONNX_ATEN_FALLBACK,
                verbose=True,  # NOTE: uncomment this for debugging
                export_params=True,
                input_names=['batched_inputs']
            )
            onnx_model = onnx.load_from_string(f.getvalue())
            onnx.save(onnx_model, "model.onnx")

    return onnx_model

In [8]:
onnx_model= export_onnx_model(model, (input_dict,))

/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/detectron2/structures/image_list.py:85: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert t.shape[:-2] == tensors[0].shape[:-2], t.shape
/home/saeed/Walnut/Quantization/ai/ditod/Wordnn_embedding.py:81: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  short_length_w = min(len(per_input_ids), len(per_input_bbox))
/home/saeed/Walnut/Quantization/ai/ditod/Wordnn_embedding.py:92: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be tre

In [ ]:
onnx_model(input_dict)

In [ ]:
import torch
# Simple module for demonstration
class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.param = torch.nn.Parameter(torch.rand(3, 4))
        self.linear = torch.nn.Linear(4, 5)

    def forward(self, x):
        return self.linear(x + self.param).clamp(min=0.0, max=1.0)

module = MyModule()

In [ ]:
inp = torch.rand(3,4)

In [ ]:
import torch.onnx as onnx
onnx.export(module, args=(inp), f="aba.onnx")

In [ ]:
import onnxruntime
inference = onnxruntime.InferenceSession("/home/saeed/Walnut/Quantization/model.onnx", None)

In [ ]:
quantized = torch.ao.quantization.quantize_dynamic(model ,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8) 

In [ ]:
quantized(image_batch, input_dict)

In [ ]:
import torch
from torch.ao.quantization import (
  get_default_qconfig_mapping,
  get_default_qat_qconfig_mapping,
  QConfigMapping,
)
import torch.ao.quantization.quantize_fx as quantize_fx
import copy


#
# post training dynamic/weight_only quantization
#

# we need to deepcopy if we still want to keep model_fp unchanged after quantization since quantization apis change the input model
model_to_quantize = copy.deepcopy(model)
model_to_quantize.eval()
qconfig_mapping = QConfigMapping().set_global(torch.ao.quantization.default_dynamic_qconfig)
# a tuple of one or more example inputs are needed to trace the model
# example_inputs = (torch.rand(3,224,224))
# prepare
example_input = (image_batch, input_dict,)
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, {"": example_input})
# no calibration needed when we only have dynamic/weight_only quantization
# quantize
model_quantized = quantize_fx.convert_fx(model_prepared)

Proxy(batched_inputs)
Proxy(getattr_1)
Proxy(size_1)


TypeError: zeros() received an invalid combination of arguments - got (tuple, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
a = list(list(model.children())[0].children())[0]

In [ ]:
a

Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))

In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
model = fasterrcnn_resnet50_fpn(pretrained=True)
dummpy_input = torch.randn(1, 3, 1333, 800)
model.eval()
model(dummpy_input)

# Export the model
torch.onnx.export(model,               # model being run
                  dummpy_input,                         # model input (or a tuple for multiple inputs)
                  "model.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'],)

/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/saeed/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:11<00:00, 14.1MB/s] 
/home/saeed/anaconda3/envs/VGT/lib/python3.10/site-packages/torch/nn/functional.py